### Problem 0: Linear Regression
Recall the linear regression problem: given a data matrix, $X$, and responses $y$, we wish to determine the model parameters $\theta^*$ that minimizes $\|X \theta - y\|_2^2$. This problem is also known as the _linear least squares_ problem.

In your classwork you learned to estimate the parameters $\theta^*$. Here you will test the significance of those parameters and report the $R^2$ value for the model. We will continue to use the same functions to generate datasets for this question.

#### Setup the workspace

In [1]:
import numpy as np
# Data and computation
import scipy as sp
import scipy.linalg
import scipy.stats as stats
import pandas as pd

# Viz
from IPython.display import display, Math
from matplotlib.pyplot import figure, subplot, xlim, ylim
from matplotlib.pyplot import scatter, axis, xlabel, ylabel, title, plot
%matplotlib inline

# Some functions we'll use later to display results
def show_cond_fancy(x, name, opt=''):
    """Display a condition number in 'fancy' format (using LaTeX)."""
    def sci_to_latex(x, fmt='{:.2e}'):
        s_raw = fmt.format(x)
        s, e = s_raw.split('e')
        return s + r'\times 10^{{{}}}'.format(int(e))
    from IPython.display import Math
    x_s = sci_to_latex(x)
    display(Math(r'\kappa({}){} \approx {}'.format(name, opt, x_s)))
    
def show_2vecs_tibble(x, y, xname='x', yname='y', error=False):
    """Display two column vectors side-by-side in a tibble."""
    assert type(x) is np.ndarray and x.ndim >= 2 and x.shape[1] == 1
    assert type(y) is np.ndarray and y.ndim >= 2 and y.shape[1] == 1
    assert x.shape == y.shape
    x_df = pd.DataFrame(x, columns=[xname])
    y_df = pd.DataFrame(y, columns=[yname])
    df = pd.concat([x_df, y_df], axis=1)
    if error:
        df['error'] = x - y
    display(df)
    
# Display (X, y) problem as a tibble
def make_data_tibble(X, y=None):
    df = pd.DataFrame(X, columns=['x_{}'.format(i) for i in range(X.shape[1])])
    if y is not None:
        y_df = pd.DataFrame(y, columns=['y'])
        df = pd.concat([y_df, df], axis=1)
    return df
    
# From: https://stackoverflow.com/questions/17129290/numpy-2d-and-1d-array-to-latex-bmatrix
def nparray_to_bmatrix(a):
    """Returns a LaTeX bmatrix"""
    assert len(a.shape) <= 2, 'bmatrix can at most display two dimensions'
    lines = str(a).replace('[', '').replace(']', '').splitlines()
    rv = [r'\begin{bmatrix}']
    rv += ['  ' + ' & '.join(l.split()) + r'\\' for l in lines]
    rv +=  [r'\end{bmatrix}']
    return '\n'.join(rv)

# Stash this function for later:
SAVE_LSTSQ = np.linalg.lstsq # You may ignore this line, which some test cells will use

def generate_model (n):
    """Returns a set of (random) n+1 linear model coefficients."""
    return np.random.rand (n+1, 1)

def generate_data (m, theta, sigma=1.0/(2**0.5)):
    """
    Generates 'm' noisy observations for a linear model whose
    predictor (non-intercept) coefficients are given in 'theta'.
    Decrease 'sigma' to decrease the amount of noise.
    """
    assert (type (theta) is np.ndarray) and (theta.ndim == 2) and (theta.shape[1] == 1)
    n = len (theta)
    X = np.random.rand (m, n)
    X[:, 0] = 1.0
    y = X.dot (theta) + sigma*np.random.randn (m, 1)
    return (X, y)

# m is the number of data points and n is the number of predictors
m = 100000
k = 5
np.random.seed(123)
theta_true = generate_model(k)
(X, y) = generate_data(m, theta_true, sigma=0.1)

**Exercise 0** (1 point): Estimate parameters

Recall that $\theta^*$ = $(X^TX)^{-1}X^Ty$

Recall how parameters were estimated using linalg and then estimate the parameters. Define the function and set the variable <mark>**"theta"**</mark> to the estimated coefficients

In [2]:
def estimate_coeffs (X, y):
    ###
    Xt_X = X.T.dot(X)
    Xt_X_inv = np.linalg.inv(Xt_X)
    Xt_y = X.T.dot(y)
    return Xt_X_inv.dot(Xt_y)
    ###

### Set theta here
###
theta = estimate_coeffs(X, y)
###


In [3]:
# Test cell: `exercise0` (1 point)
assert (np.round(theta_true - theta,1) == 0).all()
print("Passed!")

Passed!


In [4]:
theta.shape

(6, 1)

In [5]:
theta.ravel().shape

(6,)

**Exercise 1** (2 points): Calculate $R^2$.  

Also called the coefficient of determination, denoted R2 or r2 and pronounced "R squared", is the proportion of the variance in the dependent variable that is predictable from the independent variable(s). Refer to the [wiki page](https://en.wikipedia.org/wiki/Coefficient_of_determination) for more details. 

For the purpose of this question, you need to know the following terms.

$SS_{tot} = \sum\limits_{i=1}^n (y_i - \bar{y})^2$

$SS_{res} = \sum\limits_{i=1}^n (y_i - f_i)^2$

$R^2 = 1 - \frac{SS_{res}}{SS_{tot}}$

**y** is the response and **f** is the predicted value

Using the data provided above, define the three functions below and set the three values <mark>**sst, ssr, r_sq**</mark> using the respective functions.

In [6]:
def ss_total(y):
    ###
    y_mean = np.mean(y)
    diffs = np.subtract(y, y_mean)
    y_diff_sq = diffs ** 2
    ###
    return np.sum(y_diff_sq)

def ss_res(X, y, theta):
    ###
    y_hat = X.dot(theta)
    resids = np.subtract(y, y_hat)
    res_sq = resids ** 2
    return np.sum(res_sq)
    ###

def r_square(X, y, theta):
    ###
    ss_R = ss_res(X, y, theta)
    ss_T = ss_total(y)
    return 1 - (ss_R / ss_T)
    ###

### Set sst, ssr, r_sq here
###
sst = ss_total(y)
ssr = ss_res(X, y, theta)
r_sq = r_square(X, y, theta)
###


In [7]:
# Test cell: `exercise1` (2 points)
assert np.round(r_sq * ssr * sst,0) == 9459788
print("Passed!")

Passed!


**Exercise 2** (3 points): Calculate standard errors for parameters. Recall that 

$var(\hat{\theta}) = \hat{\sigma}^2 (X^TX)^{-1}$

$\hat{\sigma}^2 = \frac{1}{m-(k+1)} \sum\limits_{i=1}^m\hat\epsilon_i^2$, where 
* m is the number of data points 
* k is the number of predictors and
* epsilon is the difference between actual and predicted value (i.e. $\hat\epsilon_i=y_i - f_i$)

$se(\hat{\theta}) =$ diagonal elements of $\sqrt{var(\hat{\theta})}$  

Hint: According to above formula, the standard errors are the diagonal elments of squared root of $var(\hat{\theta})$.

Define the following function and return standard errors for parameters. Set the <mark>"serr"</mark> array with the standard errors. 

Note: return value should be an array of dimensions (k+1) X 1, where k is the number of predictors

In [8]:
def std_error(X, y, theta):
     
    ###
    m = len(X)        # Number of data points
    
    constant = (1 / (m - (k + 1)))
    
    ss_residuals = ss_res(X, y, theta)
    
    sigma_hat_sq = constant * ss_residuals
    variance = sigma_hat_sq * np.linalg.inv(X.T.dot(X))
    
    return np.diag(np.sqrt(variance)).reshape(-1, 1)
    ###

### Set "serr" here
###
serr = std_error(X, y, theta)
print(serr.shape)
print(serr)
###


(6, 1)
[[0.00127037]
 [0.00109918]
 [0.0010978 ]
 [0.00109996]
 [0.0011002 ]
 [0.00109979]]


/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:13: RuntimeWarning: invalid value encountered in sqrt
  del sys.path[0]


In [9]:
# Test cell: `exercise2` (3 points)
assert serr.shape[0] == 6
assert serr.shape[1] == 1
assert np.round(np.sum(serr),4) == 0.0068
assert (np.round(serr[1] + serr[3],5) == 0.0022)
assert (np.round(serr[2] + serr[4],5) == 0.0022)
assert (np.round(serr[5] + serr[0],5) == 0.00237)
print("Passed!")

Passed!


**Exercise 3** (2 points): Calculate  t-statisic and the corresponding p-values

$t_i = \frac{\hat{\theta_i}}{se(\hat{\theta_i})}$

$p-value = 2 * (1- P(T\leq|t_i|))$ . More details [here](https://en.wikipedia.org/wiki/P-value).

For calculating this you will need, $df ($degrees $of $freedom) = m - (k+1), where m is the number of observations and k is the number of predictors, $t_i$ is the t-statistic you calculated above.
* Set "df" to the appropriate number. 
* Define the following functions
* Use scipy stats library (stats.t.sf) and use the t-statistic (computed) to obtain the p_values. 
* Set <mark>"tstat"</mark> array with the t-statistic and <mark>"pvals"</mark> array with the p-values of the corresponding parameters. Array shape should be the same shape as that of <mark>"theta"</mark>

Note: For the functions, return valuse should be an array of shape (k+1) X 1, where k is the number of predictors

In [10]:
import scipy.stats as stats

In [11]:
theta

array([[0.69508066],
       [0.28673186],
       [0.22835872],
       [0.55018565],
       [0.7198887 ],
       [0.42433513]])

In [12]:
serr

array([[0.00127037],
       [0.00109918],
       [0.0010978 ],
       [0.00109996],
       [0.0011002 ],
       [0.00109979]])

In [13]:
theta[0] / serr[0]

array([547.14760379])

In [14]:
k

5

In [15]:
def t_statistic(theta, serr):
    ###
    return np.array([t / s for t, s in zip(theta, serr)])
    ###

def p_values(tstat, df):
    ###
    p_vals = np.zeros(tstat.shape)
    
    for i, tt in enumerate(tstat):
        t = abs(tt)
        p_vals[i] = stats.t.sf(np.abs(t), df)*2 
        
    return p_vals
    ###

# set df, tstat and pvals here
###
m = X.shape[0]
print(m)
print(k)

df = m - (k + 1)
print(df)

tstat = t_statistic(theta, serr)
print(tstat)
print(np.sum(tstat))

pvals = p_values(tstat, df)
print(pvals.shape)
print(pvals)
print(np.sum(tstat) + np.sum(pvals))
assert round(np.sum(tstat) + np.sum(pvals), 5) == 2556.3704 
###


100000
5
99994
[[547.14760379]
 [260.86093254]
 [208.01451981]
 [500.18517903]
 [654.32804571]
 [385.83411449]]
2556.3703953675326
(6, 1)
[[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]]
2556.3703953675326


In [16]:
# Test cell: `exercise3` (2 points)
assert tstat.shape[0] == 6
assert tstat.shape[1] == 1
assert pvals.shape[0] == 6
assert pvals.shape[1] == 1
assert np.round(np.sum(serr),4) == 0.0068
assert (np.round(np.sum(tstat),4) + np.sum(pvals) == 2556.3704)
assert np.round(tstat[0]+tstat[1], 5) == 808.00854
assert np.round(tstat[2]+tstat[3], 5) ==  708.1997
assert np.round(tstat[4]+tstat[5], 5) == 1040.16216

print("Passed!")

Passed!


**Exercise 4** (2 points): Find the confidence interval of regression co-efficients. 

For a given $\alpha$, confidence interval states that all possible sample estimates (for that parameter) will be in this particular interval, 100 * (1-$\alpha$)% of the time. Let's proceed to calculate the confidence intervals.

$Confidence\ Interval\ of\ \theta = \hat{\theta_i}\pm t_{\alpha/2, m-(k+1)} * se(\hat{\theta_i})$ 

We have already calculated theta and serr values above. Degrees of freedom is same as that mentioned in exercise 3. Use them to estimate the confidence intervals for parameters at <mark>$\alpha$ = 0.05</mark>. Use Scipy function ("stats.t.ppf") to get the t-value.

Return the array <mark>cinterval</mark>. Rows represent parameters and Columns represent the low-interval and high-intervals computed for that particular parameter.

Note: return value should be 2D array.

In [17]:
def conf_interval(theta, serr, df, alpha):
    ###
    conf_interval = np.zeros(shape=theta.shape)
    
#     lows_highs = []
#     for i in range(len(theta)):
    t = stats.t.ppf(alpha/2, df)
    high = theta + (t * serr)
    low = theta - (t * serr)

    return np.array(list(zip(high, low)))
    ###

### Set cinterval
###
cinterval = conf_interval(theta, serr, df, 0.05)
print(cinterval.shape)
print(np.sum(cinterval))
###


(6, 2, 1)
5.809161447038596


In [18]:
# Test cell: `exercise4` (2 points)
step1 = np.round(stats.t.ppf(1-np.abs(0.05/2), df)*2*serr,7)
step2 = np.round((cinterval[:,1] - cinterval[:,0]).reshape(6,1),7)

assert np.round(np.sum(cinterval),5) == 5.80916
assert (step1 == step2).all()
print("Passed!")

Passed!


**Great!** You have reached the end of the problem. Submit and proceed to the next problem. Don't forget to restart the kernel and run the entire notebook from top-to-bottom to make sure you did everything correctly. If that is working, try submitting this problem. (RecalL that you *must* submit and pass the autograder to get credit for your work!)

#### Optional: Fitting X and y using StatsModel (Verification)
If you are interested then you can see how close your values are to the values computed by the standard library. We print r_sq, serr, pvals, tstat and cinterval values to check against the values output by the standard library. 

In [19]:
import statsmodels.api as sm

/usr/local/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [20]:
model = sm.OLS(y, X).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.903
Model:                            OLS   Adj. R-squared:                  0.903
Method:                 Least Squares   F-statistic:                 1.869e+05
Date:                Sun, 28 Apr 2019   Prob (F-statistic):               0.00
Time:                        16:23:57   Log-Likelihood:                 88064.
No. Observations:              100000   AIC:                        -1.761e+05
Df Residuals:                   99994   BIC:                        -1.761e+05
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.6951      0.001    547.148      0.000       0.693       0.698
x1             0.2867      0.001    260.861      0.000       0.285       0.289
x2             0.2284      0.001    208.015      0.000       0.226       0.231
x3             0.5502      0.001    500.185      0.000       0.548       0.552
x4             0.7199      0.001    654.328      0.000       0.718       0.722
x5             0.4243      0.001    385.834      0.000       0.422       0.426
==============================================================================
Omnibus:                        1.886   Durbin-Watson:                   2.013
Prob(Omnibus):                  0.389   Jarque-Bera (JB):                1.875
Skew:                           0.009   Prob(JB):                        0.392
Kurtosis:                       3.010   Cond. No.                         7.97
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

#### Printing summary stats r-sq, serr, tstat, pvalues and cinterval calculated 

In [21]:
print("Regression Results")
print("R-Squared: ", np.round(r_sq,3))
final_df = pd.DataFrame(np.round(theta,4))

final_df.columns = ['coef']
final_df['std err'] = pd.DataFrame(np.round(serr,3))
final_df['t'] = pd.DataFrame(np.round(tstat,3))
final_df['P > |t|'] = pd.DataFrame(np.round(pvals,3))
final_df['[0.025'] = pd.DataFrame(np.round(cinterval[:,0], 3))
final_df['0.975]'] = pd.DataFrame(np.round(cinterval[:,1], 3))

display(final_df)

Regression Results
R-Squared:  0.903


,coef,std err,t,P > |t|,[0.025,0.975]
0,0.6951,0.001,547.148,0.0,0.693,0.698
1,0.2867,0.001,260.861,0.0,0.285,0.289
2,0.2284,0.001,208.015,0.0,0.226,0.231
3,0.5502,0.001,500.185,0.0,0.548,0.552
4,0.7199,0.001,654.328,0.0,0.718,0.722
5,0.4243,0.001,385.834,0.0,0.422,0.426


**Fin!** You've reached the end of this problem. Don't forget to restart the kernel and run the entire notebook from top-to-bottom to make sure you did everything correctly. If that is working, try submitting this problem. (RecalL that you *must* submit and pass the autograder to get credit for your work!)